In [1]:
import numpy as np
import gym
import or_gym
import ray
from ray.rllib.agents import ppo
import time
from ray import tune
from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet_v2 import FullyConnectedNetwork

In [2]:
env = gym.make('VehicleRouting-v1')

In [3]:
locs = np.repeat(56, 3)
loads = np.zeros(3)

In [5]:
np.concatenate([locs, loads, 1])

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 2 has 0 dimension(s)